# Predicting 'GPS Speed' of Truck 2

#### Imports and global variables are defined here.

In [1]:
# Imports required for this notebook.
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
import seaborn as sns

# Local path to the CSV file containing the data for truck one (1).
truckOnePath = "../data/trucks/truck1.csv"

# Local path to the CSV file containing the data for truck two (2).
truckTwoPath = "../data/trucks/truck2.csv"

#### Functions are defined here.

In [2]:
"""Uses Pandas's read_csv method to read a CSV file and returns a DataFrame of it to the notebook.
This function reads in all rows and defines the header row at index 0 by default."""
def readCsv(truck, records = None, headerIdx = 0):
    return pd.read_csv(truck, nrows = records, header = [headerIdx])

#### Predicting GPS Speed for Truck 2

Based on our previous statistics, it is clear that the GPS Speed for Truck 2 is incorrect. Perhaps the component was broken, or maybe it was set to a different unit. Either way, it is not consistent with the Wheel-Based Vehicle Speed (which appears to be working correctly) and it is not consistent with the trends found in Truck 1, where both components were measuring relatively similar speeds at any given time.

In an attempt to rectify this, we will be using data from Truck 1 and analytically predict the GPS Speed values for Truck 2 given its Wheel-Based Vehicle Speed values.

In [3]:
# Read in the data for Truck 1.
truckOneDf = readCsv(truckOnePath)
truckOneDf = truckOneDf.ffill().bfill()

# Read in the data for Truck 2.
truckTwoDf = readCsv(truckTwoPath)
truckTwoDf = truckTwoDf.ffill().bfill()